In [223]:
import numpy as np

In [224]:
from tensorflow.keras import datasets

(X_tn0, y_tn0), (X_te0, y_te0) = datasets.mnist.load_data()

In [225]:
input = X_tn0[0]
input.shape

(28, 28)

In [226]:
target = np.array(y_tn0[0]).reshape(1, -1)
target.shape

(1, 1)

In [227]:
class PADDING:

  def padding(self, input, padding_size):
    """
    input : 입력 데이터
    padding_size : padding 연산 후의 데이터 크기
    """
    if(padding_size == 0):
      return input

    padding_matrix = np.zeros((input.shape[0] + padding_size, input.shape[1] + padding_size))

    for i in range(input.shape[0]):
      for j in range(input.shape[1]):
        padding_matrix[i + int(padding_size / 2)][j + int(padding_size / 2)] = input[i][j]
    
    return padding_matrix


In [228]:
class POOLING:
  pad = PADDING()

  pool_result = []

  def max_pooling(self, input, pooling_size):
    """
    input : 입력 데이터
    pooling_size : pooling size
    """
    self.pool_result = []
    
    pooling_matrix = self.pad.padding(input, input.shape[0] % pooling_size)

    for col in range(0, pooling_matrix.shape[0], pooling_size):
      for row in range(0, pooling_matrix.shape[1], pooling_size):
        pool_arr = []
        for pooling_col in range(pooling_size):
          for pooling_row in range(pooling_size):        
            pool_arr.append(pooling_matrix[pooling_col + col, pooling_row + row])
        self.pool_result.append(max(pool_arr))
    
    # 연산 결과를 크기에 맞게 바꿔준다.
    return np.array(self.pool_result).reshape(int(pooling_matrix.shape[0] / pooling_size), -1)



In [229]:
class CNN:

  # 필터의 개수만큼 가중치가 존재
  cnn_weight = []
  
  # 패딩
  pad = PADDING()

  # 풀링
  pool = POOLING()

  # 데이터 크기
  data_size = 0

  # 레이어 구현
  layer_result = []

  # 현재 레이어 
  layer = 0
  
  def cal_cnn(self, input, target, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []
    
    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 필터 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      self.cnn_weight.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱 연산 수행
      for col in range(input.shape[0] - weight.shape[0] + 1):
        for row in range(input.shape[1] - weight.shape[1] + 1):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(input[col + w_col, row + w_row ] * weight[w_col, w_row])
          result_arr.append(np.sum(result))

      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(input.shape[0] - weight.shape[0] + 1, -1)

      # 연산 결과의 저장 (numpy 형태)
      filter_result_arr.append(result_arr)
    
    # 층에다가 결과를 저장한다. 모든 필터의 결과가 저장
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  def same_padding_cnn(self, input, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []
    
    # 데이터 크기 지정
    self.data_size = input.shape[0]

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 필터 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      self.cnn_weight.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱 연산 수행행
      for col in range(input.shape[0] - weight.shape[0] + 1):
        for row in range(input.shape[1] - weight.shape[1] + 1):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(input[col + w_col, row + w_row ] * weight[w_col, w_row])
          result_arr.append(np.sum(result))

      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(input.shape[0] - weight.shape[0] + 1, -1)

      result_arr = self.pad.padding(result_arr, input.shape[0] - result_arr.shape[0])

      # 연산 결과의 저장
      filter_result_arr.append(result_arr)
    
    # 층에다가 결과를 저장한다.
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  def pooling(self, pooling_size):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 이전 레이어의 필터 개수만큼 반복.
    for i in range(len(self.layer_result[self.layer - 1])):
      # numpy array 의 반환
      pooling_result = self.pool.max_pooling(self.layer_result[self.layer - 1][i], pooling_size)

      filter_result_arr.append(pooling_result)
    
    self.layer_result.append(filter_result_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  # 두 번째 층 이후의 cnn 연산 함수, 입력값은 이전 층의 출력이 된다.
  def layer_cnn(self, filter_size, filter_count):
    # 각 필터별 연산 결과를 저장할 리스트
    filter_result_arr = []

    # 각 layer 별 가중치를 저장할 리스트
    weight_arr = []

    # 입력 받은 필터의 개수만큼 반복
    for i in range(filter_count):

      # 필터 크기에 맞는 임의의 가중치 생성
      weight = np.random.random(filter_size * filter_size).reshape(filter_size, filter_size)
      
      weight_arr.append(weight)
      
      # 합성곱 연산 결과가 저장된다.
      result_arr = []

      # 합성곱 연산 수행. 이전 레이어의 필터 출력 크기에 맞게 반복이 시행된다.
      for col in range(self.layer_result[self.layer - 1][0].shape[0] - weight.shape[0] + 1):
        for row in range(self.layer_result[self.layer - 1][0].shape[1] - weight.shape[1] + 1):
          result = []
          for w_col in range(weight.shape[0]):
            for w_row in range(weight.shape[1]):
              result.append(input[col + w_col, row + w_row ] * weight[w_col, w_row])
          result_arr.append(np.sum(result))
          
      # 연산 결과를 크기에 맞게 바꿔준다.
      result_arr = np.array(result_arr).reshape(self.layer_result[self.layer - 1][0].shape[0] - weight.shape[0] + 1, -1)

      result_arr = self.pad.padding(result_arr, self.layer_result[self.layer - 1][0].shape[0] - result_arr.shape[0])

      # 연산 결과의 저장
      filter_result_arr.append(result_arr)
    
    # 층에다가 결과를 저장한다.
    self.layer_result.append(filter_result_arr)

    # 가중치의 저장
    self.cnn_weight.append(weight_arr)

    # 레이어의 추가
    self.layer = self.layer +1

  def flatten(self):
    data = self.layer_result[self.layer - 1]
    
    data_arr = []

    # 필터의 개수만큼 반복
    for filter_count in range(len(data)):
      for filter_size_col in range(data[filter_count].shape[0]):
        for filter_size_row in range(data[filter_count].shape[1]):
          data_arr.append(data[filter_count][filter_size_col][filter_size_row])

    return np.array(data_arr).reshape(-1,1)


In [230]:
cnn = CNN()

In [231]:
cnn.same_padding_cnn(input, 7, 64)

In [232]:
cnn.layer_result[0][0].shape

(28, 28)

In [233]:
len(cnn.layer_result[0])

64

In [234]:
cnn.pooling(2)

In [235]:
cnn.layer_result[1][0].shape

(14, 14)

In [236]:
cnn.layer_cnn(3, 128)

In [237]:
cnn.layer_result[2][0].shape

(14, 14)

In [238]:
len(cnn.layer_result[2])

128

In [239]:
cnn.pooling(2)

In [240]:
cnn.layer_result[3][0].shape

(7, 7)

In [241]:
len(cnn.layer_result[3])

128

In [242]:
cnn.layer_cnn(3, 256)

In [243]:
cnn.layer_result[4][0].shape

(7, 7)

In [244]:
len(cnn.layer_result[4])

256

In [245]:
cnn.pooling(2)

In [246]:
cnn.layer_result[5][0].shape

(4, 4)

In [247]:
len(cnn.layer_result[5])

256

In [248]:
len(cnn.layer_result[5])

256

In [249]:
cnn.flatten().shape

(4096, 1)